In [1]:
!pip install openai
!pip install openpyxl
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.1/948.1 kB 11.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [openai]2m2/3 [openai]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [1]:
import pandas as pd
import openai #for LLMs calls

ModuleNotFoundError: No module named 'openai'

In [3]:
df = pd.read_parquet(
    "https://minio.lab.sspcloud.fr/projet-formation/diffusion/mlops/data/firm_activity_data.parquet"
)
df.head()

,nace,text
0,8220Z,MISSIONS PONCTUELLES A L AIDE D UNE PLATEFORME
1,8553Z,INSPECTEUR AUTOMOBILE
2,5520Z,LA LOCATION TOURISTIQUE DE LOGEMENTS INSOLITES...
3,4791A,COMMERCE DE TOUT ARTICLES ET PRODUITS MARCHAND...
4,9499Z,REGROUPEMENT RETRAITE


In [4]:
nace_2digit_to_section = {
    # Section A: Agriculture, forestry and fishing
    "01": "A", "02": "A", "03": "A",

    # Section B: Mining and quarrying
    "05": "B", "06": "B", "07": "B", "08": "B", "09": "B",

    # Section C: Manufacturing
    "10": "C", "11": "C", "12": "C", "13": "C", "14": "C",
    "15": "C", "16": "C", "17": "C", "18": "C", "19": "C",
    "20": "C", "21": "C", "22": "C", "23": "C", "24": "C",
    "25": "C", "26": "C", "27": "C", "28": "C", "29": "C",
    "30": "C", "31": "C", "32": "C", "33": "C",

    # Section D: Electricity, gas, steam and air conditioning supply
    "35": "D",

    # Section E: Water supply; sewerage, waste management and remediation
    "36": "E", "37": "E", "38": "E", "39": "E",

    # Section F: Construction
    "41": "F", "42": "F", "43": "F",

    # Section G: Wholesale and retail trade
    "45": "G", "46": "G", "47": "G",

    # Section H: Transportation and storage
    "49": "H", "50": "H", "51": "H", "52": "H", "53": "H",

    # Section I: Accommodation and food service activities
    "55": "I", "56": "I",

    # Section J: Information and communication
    "58": "J", "59": "J", "60": "J", "61": "J", "62": "J", "63": "J",

    # Section K: Financial and insurance activities
    "64": "K", "65": "K", "66": "K",

    # Section L: Real estate activities
    "68": "L",

    # Section M: Professional, scientific and technical activities
    "69": "M", "70": "M", "71": "M", "72": "M", "73": "M", "74": "M", "75": "M",

    # Section N: Administrative and support service activities
    "77": "N", "78": "N", "79": "N", "80": "N", "81": "N", "82": "N",

    # Section O: Public administration and defence
    "84": "O",

    # Section P: Education
    "85": "P",

    # Section Q: Human health and social work activities
    "86": "Q", "87": "Q", "88": "Q",

    # Section R: Arts, entertainment and recreation
    "90": "R", "91": "R", "92": "R", "93": "R",

    # Section S: Other service activities
    "94": "S", "95": "S", "96": "S",

    # Section T: Activities of households as employers
    "97": "T", "98": "T",

    # Section U: Activities of extraterritorial organizations
    "99": "U"
}

In [5]:
df['label'] = df['nace'].apply(lambda x: x[:2]).map(nace_2digit_to_section)
df.head()

,nace,text,label
0,8220Z,MISSIONS PONCTUELLES A L AIDE D UNE PLATEFORME,N
1,8553Z,INSPECTEUR AUTOMOBILE,P
2,5520Z,LA LOCATION TOURISTIQUE DE LOGEMENTS INSOLITES...,I
3,4791A,COMMERCE DE TOUT ARTICLES ET PRODUITS MARCHAND...,G
4,9499Z,REGROUPEMENT RETRAITE,S


In [6]:
all_labels = sorted(list(set(nace_2digit_to_section.values())))
all_labels # From A to U

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U']

In [7]:
df['label'] = pd.Categorical(df['label'], categories = all_labels)
df['label'].value_counts()

label
G    122216
M    103344
I     93743
L     92525
F     61098
S     60125
H     50241
N     48516
Q     40265
J     32783
C     28091
R     26212
K     25289
P     25269
A     22429
D     12550
E      1655
O       161
B       129
U        11
T         0
Name: count, dtype: int64

In [9]:
url = "https://www.ine.es/daco/daco42/clasificaciones/cnae25/notas_explicativas_CNAE_2025.xlsx"
NAFnotes = pd.read_excel(url)
NAFnotes.head()

,Unnamed: 0,Unnamed: 1
0,NaN,Marzo de 2025
1,NaN,NaN
2,NaN,Notas explicativas CNAE-2025
3,NaN,"Incluye, incluye también y excluye por sección..."
4,NaN,NaN


In [11]:
client = openai.OpenAI(
    api_key=API_KEY,
    base_url="https://api.groq.com/openai/v1"
)

In [12]:
# Groq call
response = client.responses.create(
    model="llama-3.3-70b-versatile", # model we want to use
    input="Hi! How are you?",        # prompt
)

In [13]:
response.output_text

"Hello. I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm here and ready to help you with any questions or topics you'd like to discuss. How about you? How's your day going so far?"

In [14]:
def generate_prompt(root, text, number):
    prompt_process_notes = (
                "Vous êtes un expert en génération de données d'entraînement pour les modèles de classification.\n "
                 "Votre tâche consiste à prendre une phrase racine d'une activité économique générale et à générer des phrases d'entraînement à partir d'exemples concrets inclus dans un texte.\n "
                 "Le cas échéant, essayez d'introduire le mot 'autre' ou 'autre' ainsi que l'utilisation ou la destination du produit.\n "
                 "Chaque fois qu'un objet est fabriqué, incluez le matériau à partir duquel il est fait.\n "
                 "Évitez de produire des phrases complètement différentes ou changeant de contexte et ne retournez QUE des phrases sans introduction, commentaire, explication ou phrase supplémentaire.\n "
                 "Évitez d'introduire vos résultats par des phrases telles que 'voici les phrases générées' ou similaires.\n "
                 "Écrivez chaque phrase sur une ligne séparée.\n "
                 "Écrivez tout en majuscules.\n "
                 "Ne mets pas de point à la fin de la phrase.\n "
                 "Je veux utiliser votre résultat directement, donc je veux UNIQUEMENT les phrases.\n "
                 "Voici un exemple :\n "
                 "Racine : Production de produits de boulangerie et de confiserie.\n "
                 "Texte : Cette classe comprend la fabrication de tous les produits de boulangerie tels que : pains, gâteaux, autres farines et pains destinés à l'alimentation animale.\n "
                 "Sortie :\n "
                 "FABRICATION DE MICHES DE PAIN\n "
                 "PRODUCTION DE PAIN POUR ANIMAUX\n "
                 "PRODUCTION DE GÂTEAUX\n "
                 "PRODUCTION D'AUTRES FARINES\n "
                f"Maintenant vous le faites avec la racine et le texte suivants, en générant {number} phrases:\n "
                f"Racine: {root}.\n "
                f"Texte: {text}."
            )
    return prompt_process_notes

In [15]:
prompt = generate_prompt("Fabrication de structures métalliques et de parties de structures", 
                         """Cette sous-classe comprend :
                            - la fabrication de cadres métalliques ou d’ossatures pour la construction et de leurs éléments 
                            (tours, pylônes en treillis, armatures, ponts, éléments de ponts, etc.)
                            - la fabrication de matériel d'échafaudage, de coffrage ou d'étayage
                            - la fabrication de cadres métalliques pour équipements industriels (cadres pour hauts fourneaux, 
                            matériels de manutention, etc.)
                            - la fabrication de constructions préfabriquées principalement en métaux :
                            • baraques de chantier, éléments modulaires pour expositions, etc.
                            """, 
                         10)
print(prompt)

Vous êtes un expert en génération de données d'entraînement pour les modèles de classification.
 Votre tâche consiste à prendre une phrase racine d'une activité économique générale et à générer des phrases d'entraînement à partir d'exemples concrets inclus dans un texte.
 Le cas échéant, essayez d'introduire le mot 'autre' ou 'autre' ainsi que l'utilisation ou la destination du produit.
 Chaque fois qu'un objet est fabriqué, incluez le matériau à partir duquel il est fait.
 Évitez de produire des phrases complètement différentes ou changeant de contexte et ne retournez QUE des phrases sans introduction, commentaire, explication ou phrase supplémentaire.
 Évitez d'introduire vos résultats par des phrases telles que 'voici les phrases générées' ou similaires.
 Écrivez chaque phrase sur une ligne séparée.
 Écrivez tout en majuscules.
 Ne mets pas de point à la fin de la phrase.
 Je veux utiliser votre résultat directement, donc je veux UNIQUEMENT les phrases.
 Voici un exemple :
 Racine :

In [16]:
response = client.responses.create(
    model="llama-3.3-70b-versatile",
    input=prompt,
)

In [17]:
response.output_text.split('\n')

['FABRICATION DE CADRES MÉTALLIQUES POUR LA CONSTRUCTION',
 'FABRICATION DE TOURES EN TRÉILLIS EN MÉTAUX POUR LA CONSTRUCTION',
 'CONSTRUCTION DE PONTES EN MÉTAUX',
 "FABRICATION DE MATÉRIEL D'ÉCHAFAUDAGE EN MÉTAUX",
 'PRODUCTION DE COFFRAGE EN MÉTAUX POUR LA CONSTRUCTION',
 'FABRICATION DE CADRES MÉTALLIQUES POUR ÉQUIPEMENTS INDUSTRIELS EN ACIER',
 'CONSTRUCTION DE BARAQUES DE CHANTIER EN MÉTAUX',
 "FABRICATION D'ÉLÉMENTS MODULAIRES POUR EXPOSITIONS EN MÉTAUX",
 "PRODUCTION D'AUTRES ÉLÉMENTS DE PONTES EN MÉTAUX",
 'FABRICATION DE MATÉRIEL DE MANUTENTION EN MÉTAUX POUR ÉQUIPEMENTS INDUSTRIELS']